## Equipment Usage Metrics

This script is to be run AFTER the booking metrics script, because it uses one of the output files from that script (a CSV file) as input.


In [1]:
import requests, json, csv, calendar, datetime, math, collections, pytz, sys
from datetime import *
from os.path import exists
import pandas as pd

## INPUT variable
# Set this to the year and month for which data will be gathered, in yyy-mm-dd format. dd should always be 01
# This is also the date format LibCal's API expects for the API calls
datadate = "2024-05-01"
timeZone = pytz.timezone('Canada/Eastern')

## Lab operating days and other key values

In [2]:
# To measure equipment capacity/uage rates, we need to constrain loans to days
# when the lab is open. This is because people can niether return nor borrow
# when the pace is closed... whih means there's a variable "cap" on how
# "efficient" the quipment usage can be. The normalizes it according to
# what is available for ceckout on days when the MCL is open

# NOTE: This does not account for provincial/university holidays; some adjustment may be needed

# NOTE: This is kinda "hard-coded" for our Sun-Thurs operating range (Sept 2022-present)

[year, month, day] = datadate.split("-")
lastMonthDay = calendar.monthrange(int(year), int(month))[1]

startDate = date(int(year), int(month), int(day))
endDate = date(int(year), int(month), lastMonthDay)
print(startDate)
print(endDate)
labOperatingDays = 0

for d_ord in range(startDate.toordinal(), endDate.toordinal()):
    d = date.fromordinal(d_ord)
    if ((d.weekday() == 4) or (d.weekday() == 5)):
        labOperatingDays += 1

labOperatingDays = lastMonthDay - labOperatingDays
print(labOperatingDays)



2024-05-01
2024-05-31
23


In [3]:
# TEST to make sure the input file we need exists. If not, terminate and throw an error
# Otherwise, open and read the required input data file
if not exists("data/"+datadate+"_equip.csv"):
    sys.exit("I do not have an input file to use! Have you run the BookingsMetrics script yet?")
else:
    # Read in previously-created Equipment Data
    equipData = csv.DictReader(open("data/"+datadate+"_equip.csv"))


In [4]:
# convert date objects to datetime so we can compare in our loop
#startDate = datetime.from_date(startDate)
#endDate = datetime.from_date(endDate)
startDate = timeZone.localize(datetime.combine(startDate, datetime.min.time()))
endDate = timeZone.localize(datetime.combine(endDate, datetime.max.time()))

processedData = []

for row in equipData:
    if row['cancelled'] == 'null':
        
        # pull the loan start date and end date
        fromDate = datetime.strptime(row['fromDate'], "%Y-%m-%dT%H:%M:%S%z")
        toDate = datetime.strptime(row['toDate'], "%Y-%m-%dT%H:%M:%S%z")
    
        # check if dates are outside the month for which we're calculating things and
        # set to min/max if needed (first day of month / last day of month)
        if fromDate < startDate:
            fromDate = startDate
        if toDate > endDate:
            toDate = endDate
            
        # This totally stolen piece of code calculates te length of a loan, excpting days the lab
        # is not open (4 is Friday and 5 is Saturday)
        daygenerator = (fromDate + timedelta(x + 1) for x in range((toDate - fromDate).days))
        loanDays = sum(1 for day in daygenerator if ( (day.weekday() != 4) and (day.weekday() != 5) ))
        
        # Any "remainder" time should count as an extra day (this seems ok since due dates are always the end of the dday
        # anyway
        if (loanDays < labOperatingDays):
            loanDays += 1
        
        row['loanDays'] = loanDays
        processedData.append(row)
        

equipData = pd.DataFrame(processedData)
equipData

,bookId,id,eid,cid,lid,fromDate,toDate,created,email,status,...,project,VRexperience,flexStudioUse,flexStudioPhotoCameraChoice,flexStudioVidCameraChoice,flexStudioBackgroundChoice,groupBooking,groupSize,nickname,loanDays
0,csN5471SW,166847,22739,5659,2632,2024-04-29T11:19:00-04:00,2024-05-01T11:53:00-04:00,2024-04-28T21:16:31-04:00,76ccc04b93b27090dacdf9fd35792dca,Checked In,...,an video for business project,null,null,null,null,null,null,null,,1
1,csgnp8NSM,159683,24203,5660,2632,2024-04-24T11:13:00-04:00,2024-05-01T13:28:00-04:00,2024-03-28T14:49:44-04:00,dd5857bd6491c5f1d87cb495079d3a6f,Checked In,...,YouTube Podcast,null,null,null,null,null,null,null,,1
2,csKvPPpI1,166401,25983,5660,2632,2024-05-01T11:22:00-04:00,2024-05-01T14:51:00-04:00,2024-04-24T09:30:15-04:00,5676a61322cb2050424f700b0d50bfd6,Checked In,...,social media,null,null,null,null,null,null,null,,1
3,csKvPPpI1,166402,24944,5660,2632,2024-05-01T11:22:00-04:00,2024-05-01T14:51:00-04:00,2024-04-24T09:30:15-04:00,5676a61322cb2050424f700b0d50bfd6,Checked In,...,social media,null,null,null,null,null,null,null,,1
4,cs3qvdehM,166915,21984,5659,2632,2024-04-29T13:47:00-04:00,2024-05-01T15:47:00-04:00,2024-04-29T13:39:36-04:00,b7d30c522aeaec297d0f275c2a01e7cd,Checked In,...,Audio testing,null,null,null,null,null,null,null,,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,csAWleYta,168774,24950,5660,2632,2024-05-30T12:06:00-04:00,2024-06-10T12:28:00-04:00,2024-05-13T01:14:34-04:00,857509896c17064ea33616ec1ca395e3,Checked In,...,sports media,null,null,null,null,null,null,null,,1
249,csAWleYta,168775,22739,5659,2632,2024-05-30T12:06:00-04:00,2024-06-10T12:28:00-04:00,2024-05-13T01:14:34-04:00,857509896c17064ea33616ec1ca395e3,Checked In,...,sports media,null,null,null,null,null,null,null,,1
250,csAWleYta,168776,25984,5660,2632,2024-05-30T12:06:00-04:00,2024-06-10T12:28:00-04:00,2024-05-13T01:14:34-04:00,857509896c17064ea33616ec1ca395e3,Checked In,...,sports media,null,null,null,null,null,null,null,,1
251,csdrJzKix,170816,21668,5659,2632,2024-05-27T17:02:00-04:00,2024-06-17T17:00:00-04:00,2024-05-27T16:51:08-04:00,d143b2002c6860c3425d33b0428a5d86,Checked Out,...,Personal Work,null,null,null,null,null,null,null,,4


In [5]:
usageByBarcode = pd.pivot_table(equipData, index=['category_name','item_name','barcode'], values=['loanDays'], aggfunc=['sum','count'])
usageByBarcode


sum  \
                                                                               loanDays   
category_name   item_name                                       barcode                   
Art Tools       Wacom Intuos Medium, Bluetooth Pen Tablet       39007054933304       18   
Audio Equipment Apex176 Hypercardioid Shotgun Microphone        39007054931308       10   
                Audio-Technica AT2005USB, Cardioid Microphone   39007054933437        8   
                                                                39007054933445       11   
                                                                39007054933452        2   
...                                                                                 ...   
Video Equipment Ulanzi Phone Mount                              39007054933676        5   
                                                                39007054933684        6   
                                                                39007054933692        5   
                Vbestlife W49, Mini Dimmable LED Light Panel    39007054930607        8   
                Zhiyun Weebill 3, Gimbal for Mirrorless Cameras 39007054933296        5   

                                                                                  count  
                                                                               loanDays  
category_name   item_name                                       barcode                  
Art Tools       Wacom Intuos Medium, Bluetooth Pen Tablet       39007054933304        2  
Audio Equipment Apex176 Hypercardioid Shotgun Microphone        39007054931308        2  
                Audio-Technica AT2005USB, Cardioid Microphone   39007054933437        4  
                                                                39007054933445        4  
                                                                39007054933452        2  
...                                                                                 ...  
Video Equipment Ulanzi Phone Mount                              39007054933676        1  
                                                                39007054933684        1  
                                                                39007054933692        1  
                Vbestlife W49, Mini Dimmable LED Light Panel    39007054930607        2  
                Zhiyun Weebill 3, Gimbal for Mirrorless Cameras 39007054933296        1  

[101 rows x 2 columns]

In [6]:
usageByItemType = pd.pivot_table(equipData, index=['category_name','item_name','barcode'], values=['loanDays'], aggfunc=['sum','count'])
print(usageByItemType)

                                                                                    sum  \
                                                                               loanDays   
category_name   item_name                                       barcode                   
Art Tools       Wacom Intuos Medium, Bluetooth Pen Tablet       39007054933304       18   
Audio Equipment Apex176 Hypercardioid Shotgun Microphone        39007054931308       10   
                Audio-Technica AT2005USB, Cardioid Microphone   39007054933437        8   
                                                                39007054933445       11   
                                                                39007054933452        2   
...                                                                                 ...   
Video Equipment Ulanzi Phone Mount                              39007054933676        5   
                                                                39007054933684        6   

In [7]:
# write output to CSV in working directory
usageByBarcode.to_csv('equip_usageByBarcode_' + datadate +'.csv')
usageByItemType.to_csv('equip_usageByItemType_' + datadate +'.csv')